In [1]:
import pyomo.environ as pyomo
import pandas as pd

In [2]:
m = pyomo.ConcreteModel()

In [3]:
load=pd.read_csv('scenario1-demand.csv')
solar=pd.read_csv('solar.csv')

In [4]:
T = 24
price_grid = 5
price_renew=2
deg_cost=10
soemax=40
soemin=0
pchmax=10
pchmin=0

In [5]:
S = solar['RealPower'].values
L = load['RealPower'].values

In [6]:
times = range(T)
tplus = range(T+1)

In [7]:
m.Pgrid = pyomo.Var(times, domain=pyomo.NonNegativeReals)
m.Pexport = pyomo.Var(times, domain=pyomo.NonNegativeReals)
m.soe=pyomo.Var(tplus, domain=pyomo.NonNegativeReals)
m.pch=pyomo.Var(times, domain=pyomo.Reals)
m.pdch=pyomo.Var(times, domain=pyomo.Reals)
m.deg=pyomo.Var(times, domain=pyomo.NonNegativeReals)

In [8]:
# objective
cost = sum(price_grid*m.Pgrid[t]
            - m.Pexport[t]*price_renew
            - price_grid*m.pdch[t]
            + price_grid*m.pch[t] 
            + deg_cost*m.deg[t] for t in times)

In [9]:
cost

In [10]:
m.cost = pyomo.Objective(expr = cost, sense=pyomo.minimize)

In [11]:
# constraints
m.cons = pyomo.ConstraintList()

In [12]:
for t in times:
    m.cons.add(m.pch[t]<=pchmax)
    m.cons.add(0<=m.pch[t])
    m.cons.add(m.pdch[t]<=pchmax)
    m.cons.add(0<=m.pdch[t])
    m.cons.add(m.soe[t]<=soemax)
    m.cons.add(0<=m.soe[t])
    m.cons.add(L[t] + m.Pexport[t] + m.pch[t] == m.Pgrid[t]+S[t]+m.pdch[t]+m.deg[t])
#     m.cons.add(m.Pgrid[t]*m.Pexport[t]==0)
#     m.cons.add(m.pch[t]*m.pdch[t]==0)
    m.cons.add(m.Pexport[t]>=0)
    m.cons.add(m.Pgrid[t]>=0)
    m.cons.add(m.soe[t+1] == m.soe[t] + m.pch[t] - m.pdch[t])

m.cons.add(m.soe[0] == 10)
m.cons.add(m.Pgrid[10]==0)
m.cons.add(m.Pgrid[11]==0)
m.cons.add(m.Pgrid[12]==0)


In [13]:
print("**************")
print("gridprice",price_grid)
print("***************")
print("renewable price",price_renew)
print("***************")
print("solar generation",S)
print("**************")
print("load",L)
print("***************")


**************
gridprice 5
***************
renewable price 2
***************
solar generation [ 0.     0.     0.     0.     0.     0.     0.     0.106  7.924 19.317
 29.399 44.717 47.049 47.029 44.719 37.46  20.366  8.34   0.266  0.
  0.     0.     0.     0.   ]
**************
load [ 61.077  64.231  64.612  67.146  66.027  71.29   88.297 100.229  99.979
 101.697 105.989 118.934 124.29  121.083 129.931 119.354 124.034 125.466
 126.02  125.546 132.675 126.564 126.309 122.349]
***************


In [14]:
solver = pyomo.SolverFactory("gurobi", solver_io="python")
solver.options["NonConvex"] = 2
solver.solve(m)

{'Problem': [{'Name': 'unknown', 'Lower bound': 11757.425000000001, 'Upper bound': 11757.425000000001, 'Number of objectives': 1, 'Number of constraints': 268, 'Number of variables': 145, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 145, 'Number of nonzeros': 412, 'Sense': 1, 'Number of solutions': 3}], 'Solver': [{'Name': 'Gurobi 9.50', 'Status': 'ok', 'Wallclock time': 0.3730945587158203, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

for c in m.cons:
    print(m.cons[c].lslack(), m.cons[c].uslack())

m.display()

In [15]:
print("Total cost =", m.cost(), ".")
cols = ['TIME',
        'LOAD',
        'PGRID',
        'PEXPORT',
        'CHARGE',
        'DISCH',
        'PV',     
        'DEG',
        'SOE']

df = pd.DataFrame(columns=cols)
for t in times:
    df = df.append({
        'TIME': t,
        'LOAD': L[t],
        'PGRID': pyomo.value(m.Pgrid[t]),
        'PEXPORT': pyomo.value(m.Pexport[t]),
        'CHARGE': pyomo.value(m.pch[t]),
        'DISCH': pyomo.value(m.pdch[t]),
        'PV': S[t],
        'DEG': pyomo.value(m.deg[t]),
        'SOE': pyomo.value(m.soe[t])
    },ignore_index=True)
df

Total cost = 11757.425000000001 .
time   pgrid   pexport     SOE      CHARGE     DISCH    LOAD        pv        Deg


,TIME,LOAD,PGRID,PEXPORT,CHARGE,DISCH,PV,DEG,SOE
0,0.0,61.077,51.077,0.0,0.0,10.0,0.000,0.000000e+00,10.0
1,1.0,64.231,74.231,0.0,10.0,0.0,0.000,0.000000e+00,0.0
2,2.0,64.612,74.612,0.0,10.0,0.0,0.000,0.000000e+00,10.0
3,3.0,67.146,57.146,0.0,0.0,10.0,0.000,0.000000e+00,20.0
4,4.0,66.027,76.027,0.0,10.0,0.0,0.000,0.000000e+00,10.0
5,5.0,71.290,81.290,0.0,10.0,0.0,0.000,0.000000e+00,20.0
6,6.0,88.297,98.297,0.0,10.0,0.0,0.000,0.000000e+00,30.0
7,7.0,100.229,90.123,0.0,0.0,10.0,0.106,0.000000e+00,40.0
8,8.0,99.979,102.055,0.0,10.0,0.0,7.924,0.000000e+00,30.0
9,9.0,101.697,72.380,0.0,0.0,10.0,19.317,0.000000e+00,40.0


In [16]:
df.to_excel('scenario1.xlsx')